In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crispr-models/aot_idt.pth
/kaggle/input/crispr-models/aot_idt_weights_three.pth
/kaggle/input/crispr-models/changeseq_site_param_shap_values.pkl
/kaggle/input/crispr-models/aot_idt_three.pth
/kaggle/input/crispr-models/best_model_cnn_siteseq.pth
/kaggle/input/crispr-models/siteseq_xgb_shap_values.pkl
/kaggle/input/crispr/circleseq_all.csv
/kaggle/input/crispr/aot_idt.pth
/kaggle/input/crispr/II4.csv
/kaggle/input/crispr/crisot_fingerprint_encoding.pkl
/kaggle/input/crispr/crisot_score_param.pkl
/kaggle/input/crispr/all_off_target.csv
/kaggle/input/crispr/siteseq_model.pth
/kaggle/input/crispr/crisot_merged_xgb.pkl
/kaggle/input/crispr/guideseq.csv
/kaggle/input/crispr/changeseq.csv
/kaggle/input/crispr/surroseq.csv
/kaggle/input/crispr/test_genome.fa
/kaggle/input/crispr/hek293_off_target.csv
/kaggle/input/crispr/changeseq_model.pth
/kaggle/input/crispr/changeseq_siteseq.csv
/kaggle/input/crispr/k562_off_target.csv
/kaggle/input/crispr/siteseq.csv
/kaggle/input/crispr/cir

In [ ]:
import os
import random
import torch
import numpy as np
import copy
import pickle
seed = 12345

os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, kernel_size=1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, kernel_size=1, bias=False)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'Kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiBranchConv(nn.Module):
    def __init__(self, output_channels=16, attention=True):
        super(MultiBranchConv, self).__init__()
        
        self.branch1 = nn.Conv2d(in_channels=1, out_channels=output_channels, kernel_size=(1, 16))
        self.branch2 = nn.Conv2d(in_channels=1, out_channels=output_channels, kernel_size=(2, 16))
        self.branch3 = nn.Conv2d(in_channels=1, out_channels=output_channels, kernel_size=(3, 16))
        self.branch4 = nn.Conv2d(in_channels=1, out_channels=output_channels, kernel_size=(4, 16))
        
        self.attn = attention
        self.ca1 = ChannelAttention(output_channels)
        self.ca2 = ChannelAttention(output_channels)
        self.ca3 = ChannelAttention(output_channels)
        self.ca4 = ChannelAttention(output_channels)
        self.sa1 = SpatialAttention(kernel_size=7)
        self.sa2 = SpatialAttention(kernel_size=7)
        self.sa3 = SpatialAttention(kernel_size=7)
        self.sa4 = SpatialAttention(kernel_size=7)

    def forward(self, x):

        # Branch 1: No padding needed
        out1 = F.relu(self.branch1(x))

        # Branch 2: Pad to the right (end) so shape becomes (bs, 1, 24, 16)
        x_pad2 = F.pad(x, (0, 0, 0, 1))  # Padding only at the end (on the height dimension)
        out2 = F.relu(self.branch2(x_pad2))

        # Branch 3: Pad one row at the beginning and one at the end (bs, 1, 25, 16)
        x_pad3 = F.pad(x, (0, 0, 1, 1))  # One padding at the start and one at the end
        out3 = F.relu(self.branch3(x_pad3))

        # Branch 4: Pad two rows at the beginning and one at the end (bs, 1, 26, 16)
        x_pad4 = F.pad(x, (0, 0, 1, 2))  # One at the start, Two at the end
        out4 = F.relu(self.branch4(x_pad4))

        # Apply attention if enabled
        if self.attn:
            out1 = out1 * self.ca1(out1)
            out1 = out1 * self.sa1(out1)

            out2 = out2 * self.ca2(out2)
            out2 = out2 * self.sa2(out2)

            out3 = out3 * self.ca3(out3)
            out3 = out3 * self.sa3(out3)

            out4 = out4 * self.ca4(out4)
            out4 = out4 * self.sa4(out4)

        # Remove last dimension of size 1 (from Conv2D)
        out1 = out1.squeeze(-1)
        out2 = out2.squeeze(-1)
        out3 = out3.squeeze(-1)
        out4 = out4.squeeze(-1)

        # Transpose to shape (bs, 23, 16) for concatenation later
        out1 = out1.transpose(1, 2)
        out2 = out2.transpose(1, 2)
        out3 = out3.transpose(1, 2)
        out4 = out4.transpose(1, 2)

        # Concatenate along the last dimension to get shape (bs, 23, 64)
        output = torch.cat((out1, out2, out3, out4), dim=-1)
        
        return output


In [ ]:
import torch
import torch.nn as nn
import math

class CRISPRTransformerModel(nn.Module):
    def __init__(self, config):
        super(CRISPRTransformerModel, self).__init__()
        
        # Model parameters
        self.input_dim = 64  # Original input dimension
        self.num_layers = config.get("num_layers", 2)
        self.num_heads = config.get("num_heads", 8)
        self.dropout_prob = config["dropout_prob"]
        self.number_hidden_layers = config["number_hidder_layers"]
        self.seq_length = config.get("seq_length", 23)
        
        
        # Positional encoding
        self.pos_encoder = nn.Parameter(torch.randn(1, self.seq_length, self.input_dim))
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.input_dim,
            nhead=self.num_heads,
            dim_feedforward=self.input_dim * 4,
            dropout=self.dropout_prob,
            batch_first=True,
            norm_first=True  
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=self.num_layers
        )
        
        # Convolutional preprocessing (from original model)
        self.conv = MultiBranchConv(attention=config["attn"])
        
        # Hidden layers with residual connections
        self.hidden_layers = []
        start_size = self.seq_length*self.input_dim
        for i in range(self.number_hidden_layers):
            layer = nn.Sequential(
                nn.Linear(start_size, start_size // 2),
                nn.GELU(),  # GELU activation (often better than ReLU for transformers)
                nn.Dropout(self.dropout_prob)
            )
            self.hidden_layers.append(layer)
            start_size = start_size // 2
        self.hidden_layers = nn.ModuleList(self.hidden_layers)
        
        # Output layer with LayerNorm
        self.output = nn.Linear(start_size, 2)

    def forward(self, x, src_mask=None):
        # Apply conv layer (keeping original preprocessing)
        x = self.conv(x)  # Shape: [batch_size, seq_len, input_dim]
        
        # Add positional encoding
        x = x + self.pos_encoder
        
        # Apply transformer encoder
        x = self.transformer_encoder(x)
        
        x = x.view(x.size(0), -1)
        
        # Apply hidden layers with residual connections
        for layer in self.hidden_layers:
            x = layer(x)
        
        x = self.output(x)
        
        return x

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainerDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [ ]:
def tester(model, test_x, test_y):
    test_dataset = TrainerDataset(test_x, test_y)
    test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    model.eval()
    results = []
    true_labels = []
    with torch.no_grad():
        for test_features, test_labels in test_dataloader:
            outputs = model(test_features.to(device)).detach().to("cpu")
            results.extend(outputs)
            true_labels.extend(test_labels)
    return true_labels, results

In [ ]:
class Stats:
    def __init__(self):
        self.acc = 0
        self.pre = 0
        self.re = 0
        self.f1 = 0
        self.roc = 0
        self.prc = 0
        self.tn = 0
        self.fp = 0
        self.fn = 0
        self.tp = 0
        self.scores=[]
        self.results = []
    def print(self):
        print('Accuracy: %.4f' %self.acc)
        print('Precision: %.4f' %self.pre)
        print('Recall: %.4f' %self.re)
        print('F1 Score: %.4f' %self.f1)
        print('ROC: %.4f' %self.roc)
        print('PR AUC: %.4f' %self.prc)
        print("Confusion Matrix")
        print(self.tn, "\t", self.fp)
        print(self.fn, "\t", self.tp)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve, auc, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import torch

def eval_matrices(model, test_x, test_y, debug=True, scaler="minmax"):
    true_y, results = tester(model, test_x, test_y)  # Get true labels and logits (raw model outputs)
    
    # Convert logits to probabilities using softmax
    predictions = [torch.nn.functional.softmax(r, dim=0) for r in results]
    pred_y = np.array([y[1].item() for y in predictions])  # Probabilities of class 1
    test_y = np.array([y.item() for y in true_y])  # Convert true labels to NumPy array

    # Convert raw logits to NumPy array for scaling
    raw_logits = np.array([r[1].item() for r in results])  # Logit values for class 1

    # Apply scaling to raw logits
    if scaler == "minmax":
        scaled_scores = MinMaxScaler().fit_transform(raw_logits.reshape(-1, 1)).flatten()
    elif scaler == "standard":
        scaled_scores = StandardScaler().fit_transform(raw_logits.reshape(-1, 1)).flatten()
    else:
        scaled_scores = raw_logits  # Use raw logits directly if no scaler is specified

    # Apply threshold to classify probabilities into binary classes
    threshold = 0.5
    pred_y_list = (pred_y > threshold).astype(int)

    # Calculate confusion matrix metrics
    tn, fp, fn, tp = confusion_matrix(test_y, pred_y_list).ravel()

    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(test_y, pred_y)
    auc_score = auc(recall, precision)

    # Calculate additional statistics
    acc = accuracy_score(test_y, pred_y_list)
    pr = tp / (tp + fp) if (tp + fp) > 0 else -1
    re = tp / (tp + fn) if (tp + fn) > 0 else -1
    f1 = (2 * pr * re / (pr + re)) if (pr + re) > 0 else -1

    # Create Stats object (assuming Stats is defined elsewhere)
    stats = Stats()
    stats.acc = acc
    stats.pre = pr
    stats.re = re
    stats.f1 = f1
    stats.roc = roc_auc_score(test_y, raw_logits)  # Use raw logits for ROC AUC
    stats.prc = auc_score
    stats.tn = tn
    stats.fp = fp
    stats.fn = fn
    stats.tp = tp
    stats.scores = scaled_scores  # Use scaled logits as scores
    stats.results = results

    if debug:
        print('Accuracy: %.4f' % acc)
        print('Precision: %.4f' % pr)
        print('Recall: %.4f' % re)
        print('F1 Score: %.4f' % f1)
        print('ROC AUC: %.4f' % stats.roc)
        print('PR AUC: %.4f' % auc_score)

    return stats


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import torch
from scipy.special import softmax
import joblib

def getScore(model, test_x, test_y, scaler="softmax",T=10):
    """
    Computes scaled scores from a model without calculating additional metrics.

    Args:
        model: The trained model.
        test_x: Test input features.
        test_y: Test labels (binary).
        scaler (str): Scaling method to use ('minmax', 'standard', 'softmax', or None).

    Returns:
        np.array: Scaled scores.
    """
    # Get true labels and logits (raw model outputs)
    true_y, results = tester(model, test_x, test_y)  
    
    # Convert raw logits to NumPy array for scaling
    raw_logits = np.array([r[1].item() for r in results])  # Logit values for class 1

    # Apply scaling to raw logits
    if scaler == "minmax":
        scaler_instance = MinMaxScaler()
        scaled_scores = scaler_instance.fit_transform(raw_logits.reshape(-1, 1)).flatten()
        joblib.dump(scaler_instance, "minmax_scaler.pkl")  # Save the scaler
        print("MinMaxScaler parameters:", scaler_instance.min_, scaler_instance.scale_)
    elif scaler == "standard":
        scaled_scores = StandardScaler().fit_transform(raw_logits.reshape(-1, 1)).flatten()
    elif scaler == "softmax":
        predictions = [torch.nn.functional.softmax(r/T, dim=0) for r in results]
        scaled_scores = np.array([y[1].item() for y in predictions])
        scaler_instance = MinMaxScaler()
        # scaler_instance = joblib.load("minmax_scaler.pkl")
        scaled_scores = scaler_instance.fit_transform(scaled_scores.reshape(-1, 1)).flatten()
        joblib.dump(scaler_instance, "minmax_scaler.pkl")  # Save the scaler
        print("MinMaxScaler parameters:", scaler_instance.min_, scaler_instance.scale_)
    else:
        scaled_scores = raw_logits  # Use raw logits directly if no scaler is specified

    return scaled_scores


# Load Model

In [ ]:
model_path = '/kaggle/input/crispr-models/aot_idt_weights_three.pth'
config = {
    'num_layers': 2, 
    'num_heads': 4, 
    'number_hidder_layers': 2, 
    'dropout_prob': 0.2, 
    'batch_size': 128, 
    'epochs': 50, 
    'learning_rate': 0.001, 
    'pos_weight': 30, 
    'attn': False,
    "seq_length":20
}
model = CRISPRTransformerModel(config)
model = model.to(device)
model.load_state_dict(torch.load(model_path))



In [ ]:
def one_hot_features(df):
    # print("Generating One hot encoding features...")
    
    # Nucleotides and possible pairs
    nucleotides = ['A', 'T', 'G', 'C']
    pairs = [f'{n1}{n2}' for n1 in nucleotides for n2 in nucleotides]  # 16 possible pairs
    
    # Initialize the pairwise feature matrix (rows = positions, columns = 16 pairs)
    pairwise_features = np.zeros((len(df), 20, len(pairs)))  # (samples, positions=20, pairs=16)
    
    # Loop through each row in the DataFrame and populate the pairwise features
    for idx, row in df.iterrows():
        on_seq = row['On']
        off_seq = row['Off']
        
        for pos in range(20):  # Loop through positions 1 to 20
            pair = on_seq[pos] + off_seq[pos]  # Create the pair from the same position in both sequences
            if pair in pairs:
                pair_idx = pairs.index(pair)  # Get the index of the pair
                pairwise_features[idx, pos, pair_idx] = 1  # Set the feature value to 1
    
    # Return a DataFrame with the pairwise features
    # Reshape to (len(df), 20, 16) as the final output
    pairwise_features = pairwise_features[:,:20,:]
    return pairwise_features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def graphActiveRatio(scores, true_y, num_bins=50):
    """
    Plots a bar chart showing the percentage of 1s in true_y for each bin of scores.

    Args:
        scores (list or np.array): Predicted scores in the range [0, 1].
        true_y (list or np.array): Ground truth binary labels (0 or 1).
        num_bins (int): Number of bins for dividing the scores.
    """
    # Ensure inputs are numpy arrays
    scores = np.array(scores)
    true_y = np.array(true_y)

    # Define bin edges and initialize counts
    bins = np.linspace(0, 1, num_bins + 1)
    bin_indices = np.digitize(scores, bins, right=True)

    # Calculate the percentage of 1s for each bin
    bin_counts = np.zeros(num_bins)
    bin_positives = np.zeros(num_bins)
    for i in range(num_bins):
        bin_mask = bin_indices == i + 1
        bin_counts[i] = np.sum(bin_mask)
        bin_positives[i] = np.sum(true_y[bin_mask])

    # Avoid division by zero
    active_ratios = np.divide(bin_positives, bin_counts, where=bin_counts > 0, out=np.zeros_like(bin_counts))

    # Plot the bar chart
    bin_centers = (bins[:-1] + bins[1:]) / 2
    plt.figure(figsize=(5, 4))
    plt.bar(bin_centers, active_ratios, width=1.0 / num_bins, edgecolor='k', alpha=0.7)
    plt.xlabel('Score Ranges')
    plt.ylabel('ratio of active off-targets')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig('efficiency_barchart.png', dpi=300)

    plt.show()


In [ ]:
import pandas as pd
import numpy as np

def calculate_weights(scores ,test_y, bins):
    """
    Calculate weights (active ratios) for a binary classification problem.

    Parameters:
    - model: A trained binary classification model.
    - test_x: Features for the test dataset.
    - test_y: Labels for the test dataset.
    - bins (list): List of bin edges for score ranges (e.g., [0, 0.1, 0.3, 0.6, 1.0]).

    Returns:
    - pd.DataFrame: A DataFrame with bin ranges, counts, and active ratios.
    """

    # Create a DataFrame for analysis
    data = pd.DataFrame({
        'score': scores,
        'Active': test_y
    })

    # Bin the scores using the provided bin edges
    data['bin'] = pd.cut(data['score'], bins=bins, include_lowest=True)

    # Calculate counts for active and inactive within each bin
    bin_stats = (
    data.groupby('bin', observed=False)
    .apply(
        lambda x: pd.Series({
            'active_count': (x['Active'] == 1).sum(),
            'inactive_count': (x['Active'] == 0).sum(),
            'total_count': len(x)
        }),
        include_groups=False
    )
    .reset_index()
)


    # Calculate active ratio for each bin
    bin_stats['active_ratio'] = bin_stats['active_count'] / bin_stats['total_count']

    # Handle cases where total_count is 0 to avoid division by zero
    bin_stats['active_ratio'] = bin_stats['active_ratio'].fillna(0)

    return bin_stats




In [ ]:
import pandas as pd

# Datasets dictionary
datasets = {
    'Change-seq':'/kaggle/input/crispr/changeseq.csv',
    'Site-seq': '/kaggle/input/crispr/siteseq.csv',
    'Circle-seq': '/kaggle/input/crispr/circleseq_all.csv',
    'Guide-seq': '/kaggle/input/crispr/guideseq.csv',
    'Surro-seq': '/kaggle/input/crispr/surroseq.csv',
    'TTISS': '/kaggle/input/crispr/ttiss.csv',
}

# Initialize an empty list to store DataFrames
dataframes = []

for name, path in datasets.items():
    df = pd.read_csv(path)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True).reset_index(drop=True)
combined_df = combined_df[['On', 'Off', 'Active']]
print(f"Size before dropping duplicates: {combined_df.shape}")

# Drop duplicates
deduplicated_df = combined_df.drop_duplicates()

print(f"Size after dropping duplicates: {deduplicated_df.shape}")
print(deduplicated_df.head())

deduplicated_df = deduplicated_df.reset_index(drop=True)

In [ ]:

test_x = one_hot_features(deduplicated_df)
test_y=deduplicated_df ['Active']

In [ ]:
scores = getScore(model, test_x, test_y, T=10)

In [ ]:
 graphActiveRatio(scores, true_y=test_y, num_bins=50)

In [ ]:

bins = [0, 0.4, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8,  0.85,0.9,0.95, 1.01]
wt = calculate_weights(scores,test_y,bins)
# # print(wt)
weights = wt['active_ratio']
print(bins)
print(weights.to_numpy())



In [ ]:

with open("bin_weights.pkl", 'wb') as file:
    pickle.dump([bins, weights], file)